In [ ]:
import os
import cv2 
import numpy as np
import mtcnn
from model import InceptionResNetV2
from training import normalize,l2_normalizer
from sklearn.preprocessing import Normalizer
from scipy.spatial.distance import cosine
import pickle

In [ ]:
confidence_t=0.99
recognition_t=0.5
required_size = (160,160)

In [ ]:
face_data = '../data/lfw/'
required_shape = (160,160)
face_encoder = InceptionResNetV2()
path = '../model/keras-facenet/weights/facenet_keras_weights.h5'
face_encoder.load_weights(path)
face_detector = mtcnn.MTCNN()
lfw_dict = dict()
l2_normalizer = Normalizer('l2')

In [ ]:
all_persons = os.listdir(face_data)
p=1
for person in all_persons:
    print(person)
    print(f'Completed {p}/{len(all_persons)}')
    p+=1
    IMG_PATHS = os.listdir(os.path.join(face_data,person))
    if(len(IMG_PATHS) < 2):
        continue
    encodes = []
    for i,img_path in enumerate(IMG_PATHS):
        if(i<10):
            img = cv2.imread(os.path.join(face_data,person,img_path))
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            x = face_detector.detect_faces(img_rgb)
            if len(x) > 0:
                x1, y1, width, height = x[0]['box']
                x1, y1 = abs(x1) , abs(y1)
                x2, y2 = x1+width , y1+height
                face = img_rgb[y1:y2 , x1:x2]
            else:
                face = img_rgb
            
            face = normalize(face)
            face = cv2.resize(face, required_shape)
            face_d = np.expand_dims(face, axis=0)
            encode = face_encoder.predict(face_d)[0]
            encodes.append(encode)
    
    if encodes:
        encode = np.sum(encodes, axis=0 )
        encode = l2_normalizer.transform(np.expand_dims(encode, axis=0))[0]
        lfw_dict[person] = encode

    

In [ ]:
len(lfw_dict)

In [ ]:
correct = 0
total=0

for person in lfw_dict.keys():
    IMG_PATHS = os.listdir(os.path.join(face_data,person))
    img_path = os.path.join(face_data,person,IMG_PATHS[-1])
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = face_detector.detect_faces(img_rgb)
    names =[]
    for res in results:
        if res['confidence'] > confidence_t:
            continue
    
        x1, y1, width, height = x[0]['box']
        x1, y1 = abs(x1) , abs(y1)
        x2, y2 = x1+width , y1+height
        face = img_rgb[y1:y2 , x1:x2]
        
        face = normalize(face)
        face = cv2.resize(face, required_shape)
        face_d = np.expand_dims(face, axis=0)
        encode = face_encoder.predict(face_d)[0]
        encode = l2_normalizer.transform(encode.reshape(1, -1))[0]
        
        name = 'unknown'
        distance = float("inf")
        
        for db_name, db_encode in lfw_dict.items():
            dist = cosine(db_encode, encode)
            if dist < recognition_t and dist < distance:
                name = db_name
                distance = dist
        names.append(name)
        
    if person in names:
        correct+=1
    total+=1